In [1]:
import numpy as np
class AZ():
    def __init__(self,train,test):
        self.train=train
        self.test=test
        
class Train():
    def __init__(self,images,labels):
        self.images=images
        self.labels=labels
    def next_batch(self,batch_size=15):
        idx = np.arange(0 , len(self.images))
        np.random.shuffle(idx)
        idx = idx[:batch_size]
        images_shuffle = [self.images[ i] for i in idx]
        labels_shuffle = [self.labels[ i] for i in idx]
        return (np.asarray(images_shuffle), np.asarray(labels_shuffle))

class Test():            
    def __init__(self,images,labels):
        self.images=images
        self.labels=labels

In [2]:
import pandas as pd
data=pd.read_csv('C:\Files\Datasets\A_Z HandwrittenData.csv')
print(data.shape)#28*28 pixel +1 label columns and n instances
X, y=data.iloc[:,1:].values,data.iloc[:,0].values

(372037, 785)


In [3]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [4]:
from sklearn.preprocessing import StandardScaler
#Normalize pixel values
scaler=StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

C:\Users\vikhy\Anaconda3\Anaconda-3.5\lib\site-packages\sklearn\utils\validation.py:475: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)


In [5]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
#Encode the labels using one hot encoder
labelencoder_y = LabelEncoder()
y_train = labelencoder_y.fit_transform(y_train[:])
y_test = labelencoder_y.fit_transform(y_test[:])
onehotencoder = OneHotEncoder(categorical_features = [0])
y_test = onehotencoder.fit_transform(y_test.reshape(-1,1)).toarray()
y_train = onehotencoder.fit_transform(y_train.reshape(-1,1)).toarray()

In [6]:
Az=AZ(Train(X_train,y_train),Test(X_test,y_test))

In [ ]:
import os
import os.path as path
import tensorflow as tf
from tensorflow.python.tools import freeze_graph
from tensorflow.python.tools import optimize_for_inference_lib

tf.reset_default_graph()

MODEL_NAME = 'az_mnist_convnet'
NUM_STEPS = 6000
BATCH_SIZE = 15

def model_input(input_node_name, keep_prob_node_name):
    x = tf.placeholder(tf.float32, shape=[None, 28*28], name=input_node_name)
    keep_prob = tf.placeholder(tf.float32, name=keep_prob_node_name)
    y_ = tf.placeholder(tf.float32, shape=[None, 26])
    return x, keep_prob, y_


def build_model(x, keep_prob, y_, output_node_name):
    x_image = tf.reshape(x, [-1, 28, 28, 1])
    # 28*28*1

    conv1 = tf.layers.conv2d(x_image, 64, 3, 1, 'same', activation=tf.nn.relu)
    # 28*28*64
    pool1 = tf.layers.max_pooling2d(conv1, 2, 2, 'same')
    # 14*14*64

    conv2 = tf.layers.conv2d(pool1, 128, 3, 1, 'same', activation=tf.nn.relu)
    # 14*14*128
    pool2 = tf.layers.max_pooling2d(conv2, 2, 2, 'same')
    # 7*7*128

    conv3 = tf.layers.conv2d(pool2, 256, 3, 1, 'same', activation=tf.nn.relu)
    # 7*7*256
    pool3 = tf.layers.max_pooling2d(conv3, 2, 2, 'same')
    # 4*4*256

    flatten = tf.reshape(pool3, [-1, 4*4*256])
    fc = tf.layers.dense(flatten, 1024, activation=tf.nn.relu)
    dropout = tf.nn.dropout(fc, keep_prob)
    logits = tf.layers.dense(dropout, 26)
    outputs = tf.nn.softmax(logits, name=output_node_name)

    # loss
    loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=logits))

    # train step
    train_step = tf.train.AdamOptimizer(1e-4).minimize(loss)

    # accuracy
    correct_prediction = tf.equal(tf.argmax(outputs, 1), tf.argmax(y_, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

    #Prediction node
    prediction = tf.argmax(outputs,1)
    
    tf.summary.scalar("loss", loss)
    tf.summary.scalar("accuracy", accuracy)
    merged_summary_op = tf.summary.merge_all()

    return train_step, loss, accuracy, merged_summary_op,outputs, prediction

def train(x, keep_prob, y_, train_step, loss, accuracy,
        merged_summary_op, saver,prediction):
    print("Training start...")

    hs_mnist=Az

    init_op = tf.global_variables_initializer()

    with tf.Session() as sess:
        sess.run(init_op)

        tf.train.write_graph(sess.graph_def, 'out',
            MODEL_NAME + '.pbtxt', True)

        # op to write logs to Tensorboard
        summary_writer = tf.summary.FileWriter('logs/',
            graph=tf.get_default_graph())

        for step in range(NUM_STEPS):
            batch = hs_mnist.train.next_batch(BATCH_SIZE)#mnist.train.next_batch(15)
            if step % 100 == 0:
                train_accuracy = accuracy.eval(feed_dict={
                    x: batch[0], y_: batch[1], keep_prob: 1.0})
                print('step %d, training accuracy %f' % (step, train_accuracy))
            _, summary = sess.run([train_step, merged_summary_op],
                feed_dict={x: batch[0], y_: batch[1], keep_prob: 0.5})
            summary_writer.add_summary(summary, step)

        saver.save(sess, 'out/' + MODEL_NAME + '.chkp')

        test_accuracy = accuracy.eval(feed_dict={x: hs_mnist.test.images,
                                    y_: hs_mnist.test.labels,
                                    keep_prob: 1.0})
        print('test accuracy %g' % test_accuracy)

    print("training finished!")
    return hs_mnist.test,init_op
    
def export_model(input_node_names, output_node_name):
    freeze_graph.freeze_graph('out/' + MODEL_NAME + '.pbtxt', None, False,
        'out/' + MODEL_NAME + '.chkp', output_node_name, "save/restore_all",
        "save/Const:0", 'out/frozen_' + MODEL_NAME + '.pb', True, "")

    input_graph_def = tf.GraphDef()
    with tf.gfile.Open('out/frozen_' + MODEL_NAME + '.pb', "rb") as f:
        input_graph_def.ParseFromString(f.read())

    output_graph_def = optimize_for_inference_lib.optimize_for_inference(
            input_graph_def, input_node_names, [output_node_name],
            tf.float32.as_datatype_enum)

    with tf.gfile.FastGFile('out/opt_' + MODEL_NAME + '.pb', "wb") as f:
        f.write(output_graph_def.SerializeToString())

    print("graph saved!")


In [ ]:
if not path.exists('out'):
    os.mkdir('out')

input_node_name = 'input'
keep_prob_node_name = 'keep_prob'
output_node_name = 'output'

x, keep_prob, y_ = model_input(input_node_name, keep_prob_node_name)

train_step, loss, accuracy, merged_summary_op,outputs,prediction = build_model(x, keep_prob,y_, output_node_name)
saver = tf.train.Saver()
import time 
start=time.time()
batch_test,init_op=train(x, keep_prob, y_, train_step, loss, accuracy,merged_summary_op, saver,prediction)
print('TIme elapsed:',time.time()-start)
export_model([input_node_name, keep_prob_node_name], output_node_name)

Training start...
step 0, training accuracy 0.000000
step 100, training accuracy 0.200000
step 200, training accuracy 0.666667
step 300, training accuracy 0.866667
step 400, training accuracy 0.866667
step 500, training accuracy 0.866667
step 600, training accuracy 0.733333
step 700, training accuracy 0.933333
step 800, training accuracy 0.933333
step 900, training accuracy 0.733333
step 1000, training accuracy 0.933333
step 1100, training accuracy 1.000000
step 1200, training accuracy 0.866667
step 1300, training accuracy 1.000000
step 1400, training accuracy 0.866667
step 1500, training accuracy 1.000000
step 1600, training accuracy 0.800000
step 1700, training accuracy 1.000000
step 1800, training accuracy 0.933333
step 1900, training accuracy 1.000000
step 2000, training accuracy 0.933333
step 2100, training accuracy 1.000000
step 2200, training accuracy 1.000000
step 2300, training accuracy 1.000000
step 2400, training accuracy 0.866667
step 2500, training accuracy 1.000000
step 2